In [107]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd /content/drive/MyDrive/Colab Notebooks/Kb model

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Kb model


In [108]:
# !pip install openai
# !pip install sentence_transformers
# !pip install python-dotenv
# !pip install python-multipart
# !pip install pytesseract

git fetch origin
git checkout 21-featuregpt_recommandation_wcustomer

In [1]:
import random
import json
import pandas as pd
import numpy as np
from ast import literal_eval
from typing import List

import openai
import os
import sys
from dotenv import load_dotenv

from urllib import request
from urllib.request import Request, urlopen
from PIL import Image
import pytesseract
from io import BytesIO

from ast import literal_eval
import re
from tqdm.notebook import tqdm

import torch
from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer, util
import json

from openai import OpenAI
OPENAI_API_KEY = "sk-proj-AO3KelC1emoiTrtYed3jT3BlbkFJVuDMTmSkpm2J4S7me2nO"
client = OpenAI(
  api_key=OPENAI_API_KEY
)

/var/folders/10/1jlz9hbj355b5ysnf08kr7nw0000gn/T/ipykernel_84269/1466827135.py:24: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


### 1. df_emb_total_2.csv : 데이터 전처리, Embedding vector 추출 완료된 데이터 파일

### 데이터 로드

In [2]:
# Load the JSON data into a variable named 'customer_data' to avoid overwriting
with open('output2.json', 'r', encoding='utf-8') as f:
    customer_data = json.load(f)

# JSON 데이터를 id를 키로 사용하는 딕셔너리로 변환
customer_dict = {customer['id']: customer for customer in customer_data}

In [3]:
df = pd.read_csv('./all_events_data.csv', sep = ',', dtype = str, engine='python', on_bad_lines='skip')
df['openai_emb'] = df['openai_emb'].apply(lambda x: literal_eval(x))

### 2. 다음 코드부터는 코사인 유사도 구현 후 chatgpt 이용해서 프롬프트 구현까지 진행

In [5]:
def get_query_sim_top_k(query, top_k):
    openai_model = 'text-embedding-ada-002'
    query_encode = client.embeddings.create(model = openai_model, input=query)
    query_encode = query_encode.data[0].embedding
    cos_scores = util.pytorch_cos_sim(query_encode, df['openai_emb'])[0]
    top_results = torch.topk(cos_scores, k=top_k)
    print(top_results)

    top_result_id = top_results.indices.tolist()
    top_result_score = top_results.values.tolist()

    result_df = df[df.index.isin(top_result_id)]
    result_df.index = top_result_id
    result_df['score'] = top_result_score
    answer = result_df[['score','번호','제목', '이벤트 기간', 'img', 'img_text']]

    return top_results #answer

### 3. Prompt 설정

In [10]:
#user_msg_history = []
def get_chatgpt_msg(msg):
    response = client.chat.completions.create(
                    model="gpt-3.5-turbo",
                    messages=msg
                    )
    return response.choices[0].message.content

def set_prompt(intent, query, msg_prompt_init):
    '''prompt 형태를 만들어주는 함수'''
    m = dict()
    # 검색 또는 추천이면
    if ('recom' in intent) or ('search' in intent):
        msg = msg_prompt_init['recom'] # 시스템 메세지를 가지고오고
    # 설명문이면
    elif 'desc' in intent:
        msg = msg_prompt_init['desc'] # 시스템 메세지를 가지고오고
    # intent 파악
    else:
        msg = msg_prompt_init['intent']
        msg['user'] += f' {query} \n A:'
    for k, v in msg.items():
        m['role'], m['content'] = k, v
    return [m]

In [13]:
def recommend_result(query):
    user_msg_history = []
    msg_prompt_init = {
        'recom' : {
                    'system' : "당신은 사용자의 질문에 따라 은행 이벤트를 추천하는 유용한 도우미입니다.",
                    'user' : "'물론이죠!'로 시작하는 간단한 인사말을 한 문장씩 작성해 보세요. 사용자에게 은행 이벤트 항목을 추천합니다.",
                  },
        'desc' : {
                    'system' : "당신은 친절하게 답변해주시는 유용한 도우미입니다.",
                    'user' : "사용자에게 해당 항목을 설명하기 위해 '물론이죠!'로 시작하는 간단한 인사말을 작성해주세요.",
                  },
        'intent' : {
                    'system' : "당신은 사용자의 질문 의도를 이해하는 유용한 도우미입니다.",
                    'user' : "다음 문장은 '설명', '추천', '검색' 중 어떤 카테고리에 속합니까? 카테고리만 표시합니다. \n content:"
                    }
    }


    # 1. 사용자의 의도를 파악
    user_intent = set_prompt('intent', query, msg_prompt_init)
    user_intent = get_chatgpt_msg(user_intent)
    #print("user_intent : ", user_intent)

    # 2. 사용자의 쿼리에 따라 prompt 생성
    intent_data = set_prompt(user_intent, query, msg_prompt_init)
    intent_data_msg = get_chatgpt_msg(intent_data).replace("\n", "").strip()
    #print("intent_data_msg : ", intent_data_msg)

    # 3. 추천 또는 검색이면
    if ('추천' in user_intent) or ('검색' in user_intent):
        recom_msg = str()
        # 기존에 메세지가 있으면 쿼리로 대체
        if (len(user_msg_history) > 0 ) and (user_msg_history[-1]['role'] == 'assistant'):
            query = user_msg_history[-1]['content']['feature']

        # 유사 아이템 top_k = 3개 가져오기
        top_result = get_query_sim_top_k(query, top_k=3)

        # 검색이면, 자기 자신의 컨텐츠는 제외
        top_index = top_result[1].numpy() #if '추천' in user_intent else top_result[1].numpy()[1:]

        # '번호','제목', '이벤트 기간', 'img', 'img_text'를 가져와서 출력
        r_set_d = df.iloc[top_index, :][['번호','제목', '이벤트 기간', '알아두세요', 'img', 'img_text']]
        r_set_d = r_set_d.rename(columns = {'번호':'event_id', '제목':'event_title', '이벤트 기간':'event_period',
                                            '알아두세요':'event_description', 'img':'event_img', 'img_text':'event_img_ocr'})
        r_set_d = json.loads(r_set_d.to_json(orient="records"))
        for r in r_set_d:
            for _, v in r.items():
                recom_msg += f"{v} \n"
            recom_msg += "\n"
        user_msg_history.append({'role' : 'assistant', 'content' : f"{intent_data_msg} {str(recom_msg)}"})
        #print(f"\n recom data : {intent_data_msg} {str(recom_msg)}")
        return r_set_d

### 4. query : 질문(=고객 정보) 넣어 추천 결과 도출

In [14]:
# 특정 고객의 id로 고객 정보 호출
selected_id = random.randint(1, 2000)
selected_customer = customer_dict.get(selected_id)
query = f"""아래 고객 정보 (ID: {selected_id})를 기반으로 적절한 금융 이벤트를 추천해 주세요.
{selected_customer}
"""
print(query)

아래 고객 정보 (ID: 1113)를 기반으로 적절한 금융 이벤트를 추천해 주세요.
{'id': 1113, '자치구명': '서울시 은평구', '생년월일': 1962, '성별': '남', '직업군': '일반직장인', '평균_통화량': 990.0, '대출여부': 1.0, '추정연소득_백분위': 15.0, '총대출약정금액': 100000.0, '신용대출약정금액': 70000.0, 'Top3_소비_키워드': ['가정생활/서비스', '소형유통점', '의료']}



In [15]:
#  query = '커피와 관련된 이벤트'
answer = recommend_result(query)
answer[0] #첫번째 추천 결과

torch.return_types.topk(
values=tensor([0.8275, 0.8252, 0.8212]),
indices=tensor([262, 240, 337]))


/var/folders/10/1jlz9hbj355b5ysnf08kr7nw0000gn/T/ipykernel_84269/1429551103.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result_df['score'] = top_result_score


{'event_id': '316',
 'event_title': "'한국씨티은행 신용대출 보유 고객님’만을 위한 WELCOME 이벤트",
 'event_period': '2022.11.11 ~ 2022.12.31',
 'event_description': '본 이벤트 경품지급대상은 기간 내 [이벤트 응모] 및 [KB 제휴 대환 신용대출 실행 완료] 고객 中 대출실행 합산금액 3천만원 이상 고객입니다.이벤트 기간 내 실행된 대출합산금액을 기준으로 구간별 경품이 지급됩니다.대환대출 여러 건 실행시에도 응모는 최대 1번 가능하며, 통장자동대출 계좌, 경품지급일 기준 연체계좌 / 대출철회계좌 / 전액상환계좌인 경우 경품지급대상에서 제외됩니다.이벤트 경품은 경품지급일에 약정된 본인명의 자동이체 계좌로 입금됩니다.KB 제휴 대환 신용대출 이벤트는 KB국민은행의 사정에 따라 변경 또는 중단될 수 있으며 이벤트 경품은 사정상 동일금액대의 다른 상품으로 변경될 수 있습니다.경품 지급일까지 입금 처리 불가 사유(계좌 해지, 자동이체 계좌 변경 등) 발생, 경품 수령거부 등으로 경품 수령이 불가능할 경우 당첨이 무효 처리됩니다. 경품 수령을 위한 고객정보상의 연락처를 확인하여 주시기 바랍니다. 내 고객정보 바로가기 >5만원 초과 경품 당첨시 제세공과금은 고객 부담입니다.경품 당첨자는 KB스타뱅킹 앱을 통해 확인이 가능합니다. (경로: KB스타뱅킹 > 생활/혜택 > 이벤트 > 응모/당첨확인 > 당첨 이벤트 보기 > 해당이벤트 클릭)이벤트에 관한 기타 자세한 내용은 KB국민은행 고객센터(☏1588-9999)로 문의하시기 바랍니다.본 광고물은 이벤트 관련 홍보물이며 실제 상품의 계약은 대출거래약정서, 여신거래기본약관 등의 적용을 받기 때문에 대출계약을 체결하기 전에 반드시 상품설명서 및 관련 약관을 읽어보시기 바랍니다. 약관은 창구 및 KB국민은행 홈페이지(www.kbstar.com:전체서비스 > 고객센터 > 서식/약관/설명서)에서 교부 및 열람이 가능합니다.[대출상품 체결시 유의사항

In [18]:
answer

[{'event_id': '316',
  'event_title': "'한국씨티은행 신용대출 보유 고객님’만을 위한 WELCOME 이벤트",
  'event_period': '2022.11.11 ~ 2022.12.31',
  'event_description': '본 이벤트 경품지급대상은 기간 내 [이벤트 응모] 및 [KB 제휴 대환 신용대출 실행 완료] 고객 中 대출실행 합산금액 3천만원 이상 고객입니다.이벤트 기간 내 실행된 대출합산금액을 기준으로 구간별 경품이 지급됩니다.대환대출 여러 건 실행시에도 응모는 최대 1번 가능하며, 통장자동대출 계좌, 경품지급일 기준 연체계좌 / 대출철회계좌 / 전액상환계좌인 경우 경품지급대상에서 제외됩니다.이벤트 경품은 경품지급일에 약정된 본인명의 자동이체 계좌로 입금됩니다.KB 제휴 대환 신용대출 이벤트는 KB국민은행의 사정에 따라 변경 또는 중단될 수 있으며 이벤트 경품은 사정상 동일금액대의 다른 상품으로 변경될 수 있습니다.경품 지급일까지 입금 처리 불가 사유(계좌 해지, 자동이체 계좌 변경 등) 발생, 경품 수령거부 등으로 경품 수령이 불가능할 경우 당첨이 무효 처리됩니다. 경품 수령을 위한 고객정보상의 연락처를 확인하여 주시기 바랍니다. 내 고객정보 바로가기 >5만원 초과 경품 당첨시 제세공과금은 고객 부담입니다.경품 당첨자는 KB스타뱅킹 앱을 통해 확인이 가능합니다. (경로: KB스타뱅킹 > 생활/혜택 > 이벤트 > 응모/당첨확인 > 당첨 이벤트 보기 > 해당이벤트 클릭)이벤트에 관한 기타 자세한 내용은 KB국민은행 고객센터(☏1588-9999)로 문의하시기 바랍니다.본 광고물은 이벤트 관련 홍보물이며 실제 상품의 계약은 대출거래약정서, 여신거래기본약관 등의 적용을 받기 때문에 대출계약을 체결하기 전에 반드시 상품설명서 및 관련 약관을 읽어보시기 바랍니다. 약관은 창구 및 KB국민은행 홈페이지(www.kbstar.com:전체서비스 > 고객센터 > 서식/약관/설명서)에서 교부 및 열람이 가능합니다.[대출상품 체결시 